# Messaging


### Imports

In [ ]:
from aries_cloudcontroller import AriesAgentController
import os
from termcolor import colored
import asyncio


from libs.messaging_service import MessagingService

### Initialise the Agent Controller

In [ ]:
api_key = os.getenv("ACAPY_ADMIN_API_KEY")
admin_url = os.getenv("ADMIN_URL")

print(f"Initialising a controller with admin api at {admin_url} and an api key of {api_key}")
agent_controller = AriesAgentController(admin_url,api_key)

### Start a Webhook Server

In [ ]:
webhook_port = int(os.getenv("WEBHOOK_PORT"))
webhook_host = "0.0.0.0"

await agent_controller.init_webhook_server(webhook_host, webhook_port)

print(f"Listening for webhooks from agent at http://{webhook_host}:{webhook_port}")

## Instatiate Messaging Service

In [ ]:
messaging_service = MessagingService(agent_controller)

## Optional - Configure a Default Mediator

We have hosted an ACA agent in open mediation mode. Accept this connection and request mediation from it and it should grant your request. If you set this as your default mediation all future invites will have serviceEndpoints pointing to this agent which will then forward messages on to your endpoint.

In [ ]:
mediator_invite = {'@type': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/connections/1.0/invitation', '@id': 'bb47c3ea-ab9a-4ef4-984d-409cb4909956', 'recipientKeys': ['HkwRkbCGEXn4LviXNen5f8kqQ1WxVRpQQGxaQG7PuSZD'], 'serviceEndpoint': 'http://139.162.224.50:6666', 'label': 'HGF_ACA_Playground_Mediator'}

In [ ]:
invitation_response = await agent_controller.connections.receive_invitation(mediator_invite)

In [ ]:
connection_id = invitation_response["connection_id"]
### check state of connection
connection = await agent_controller.connections.get_connection(connection_id)

# Let's check for the state
if connection['state'] != 'active':
    print("No active connection. \n Please go back and ensure you have established an active connection")    
else:
    ## request mediation
    mediation_req = await agent_controller.mediation.request_mediation(connection_id)
    

In [ ]:
### To have a look at the mediation records and we should see our mediation in there
records_response = await agent_controller.mediation.get_mediation_record_by_id(mediation_req["mediation_id"])
print(f"State : {records_response['state']}")

In [ ]:
default_mediation_res = await agent_controller.mediation.set_default_mediator(records_response['mediation_id'])
print(default_mediation_res)

## Add Contact

### Either from Invitation Shared

In [ ]:
invitation = {"@type": "did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/connections/1.0/invitation", "@id": "b352adae-24fe-44d0-a6dc-43c853242d17", "serviceEndpoint": "https://3a8742dc608e.ngrok.io", "label": "Hyperledger Global Forum", "recipientKeys": ["JE9d6umnBNqrdftsa9nLG2ExkzjdyVPrG9ghGMZ7UNxP"]}


In [ ]:
new_contact_id = messaging_service.accept_contact_invitation(invitation, "HLGF")

### Or by Creating and Sharing an Invite

In [ ]:
alias = "HLGF"
response = messaging_service.new_contact_invite(alias)

# If you want to display as Q.R for example
invitation = response["invitation"]

new_contact_id = response["connection_id"]

## Send Message to Contact by Connection ID

In [ ]:
some_arbitrary_sized_message = "Hey how are you, want to grab a coffee?"
messaging_service.send_message(new_contact_id, some_arbitrary_sized_message)

## Display Inbox for Contact

In [ ]:
messaging_service.display_inbox_for_contact(new_contact_id)

## Get Contacts

In [ ]:
messaging_service.view_inbox(show_inactive=True)

In [ ]:
response = messaging_service.get_contacts()
for contact in response:
    print(f"{contact.alias} : {contact.connection_id}")
    print(contact.is_active.done())
    print(contact.__dict__)

## Terminate Controller

Whenever you have finished with this notebook, be sure to terminate the controller. This is especially important if your business logic runs across multiple notebooks.

In [ ]:
await agent_controller.terminate()